In [2]:
# Import Libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import matplotlib.colors as colors 
import os
import snappy
from snappy import Product
from snappy import ProductIO 
from snappy import ProductUtils 
from snappy import WKTReader 
from snappy import HashMap
from snappy import GPF
# For shapefiles
import shapefile 
import pygeoif
import jpy

path_to_sentinel_data = "S1A_IW_GRDH_1SDV_20180415T163146_20180415T163211_021480_025003_8E79.SAFE/manifest.safe"
product = ProductIO.readProduct(path_to_sentinel_data)

INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Incompatible GDAL 3.3.2 found on system. Internal GDAL 3.0.0 from distribution will be used.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.


Product information

In [3]:
width = product.getSceneRasterWidth() 
print("Width: {} px".format(width)) 
height = product.getSceneRasterHeight() 
print("Height: {} px".format(height)) 
name = product.getName()
print("Name: {}".format(name))
band_names = product.getBandNames()
print("Band names: {}".format(", ".join(band_names)))

Width: 25220 px
Height: 16774 px
Name: S1A_IW_GRDH_1SDV_20180415T163146_20180415T163211_021480_025003_8E79
Band names: Amplitude_VH, Intensity_VH, Amplitude_VV, Intensity_VV


Show product inline

In [10]:
def plotBand(product, band, vmin, vmax):

    band = product.getBand(band) 
    w = band.getRasterWidth()
    h = band.getRasterHeight() 
    print(w, h)

    band_data = np.zeros(w * h, np.float32) 
    band.readPixels(0, 0, w, h, band_data)

    band_data.shape = h, w

    width = 12
    height = 12
    plt.figure(figsize=(width, height))
    imgplot = plt.imshow(band_data, cmap=plt.cm.binary, vmin=vmin, vmax=vmax)
    
    return imgplot

## Image pre-processing 
### Orbit file application
Before any SAR pre-processing steps occur, the product subset should be properly orthorectified 
to improve accuracy. To properly orthorectify the image, the orbit file is applied using the 
Apply-Orbit-File GPF module. SNAP is able to generate and apply a high accuracy satellite 
orbit file to a product.

In [4]:
parameters = HashMap() 
GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

parameters.put('Apply-Orbit-File', True)
parameters.put('orbitType', 'Sentinel Precise (Auto Download)') 
parameters.put('polyDegree', '3') 
parameters.put('continueOnFail', 'false')

apply_orbit_file = GPF.createProduct('Apply-Orbit-File', parameters, product)

INFO: org.hsqldb.persist.Logger: dataFileCache open start
INFO: org.esa.snap.engine_utilities.download.DownloadableContentImpl: http retrieving http://step.esa.int/auxdata/orbits/Sentinel-1/POEORB/S1A/2018/04/S1A_OPER_AUX_POEORB_OPOD_20180505T120640_V20180414T225942_20180416T005942.EOF.zip


100% done.


### Adding a shape file

In [6]:
r = shapefile.Reader("island_boundary/island_boundary2.shp")
g=[]
for s in r.shapes(): 
    g.append(pygeoif.geometry.as_shape(s))

m = pygeoif.MultiPoint(g)

wkt = str(m.wkt).replace("MULTIPOINT", "POLYGON(") + ")"

## Try again :(

In [24]:
import os
import datetime
import gc
import glob
import snappy
from sentinelsat import SentinelAPI, geojson_to_wkt, read_geojson
from snappy import ProductIO

In [42]:
class sentinel1_download_preprocess():
    def __init__(self, input_dir, date_1, date_2, query_style, footprint, lat=24.84, lon=90.43, download=False):
        self.input_dir = input_dir
        self.date_start = datetime.datetime.strptime(date_1, "%d%b%Y")
        self.date_end = datetime.datetime.strptime(date_2, "%d%b%Y")
        self.query_style = query_style
        self.footprint = "POLYGON ((5.712890625 61.990587736204105, 11.9091796875 61.990587736204105, 11.9091796875 64.64270382119372, 5.712890625 64.64270382119372, 5.712890625 61.990587736204105))"#geojson_to_wkt(read_geojson(footprint))
        self.lat = lat
        self.lon = lon
        self.download = download

        # configurations
        self.api = SentinelAPI('renateask', '071198Ra', 'https://scihub.copernicus.eu/dhus')
        self.producttype = 'GRD'  # SLC, GRD, OCN
        self.orbitdirection = 'ASCENDING'  # ASCENDING, DESCENDING
        self.sensoroperationalmode = 'IW'  # SM, IW, EW, WV

    def sentinel1_download(self):
        global download_candidate
        if self.query_style == 'coordinate':
            download_candidate = self.api.query('POINT({0} {1})'.format(self.lon, self.lat),
                                                date=(self.date_start, self.date_end),
                                                producttype=self.producttype,
                                                orbitdirection=self.orbitdirection,
                                                sensoroperationalmode=self.sensoroperationalmode)
        elif self.query_style == 'footprint':
            download_candidate = self.api.query(self.footprint,
                                                date=(self.date_start, self.date_end),
                                                producttype=self.producttype,
                                                orbitdirection=self.orbitdirection,
                                                sensoroperationalmode=self.sensoroperationalmode)
        else:
            print("Define query attribute")

        title_found_sum = 0
        for key, value in download_candidate.items():
            for k, v in value.items():
                if k == 'title':
                    title_info = v
                    title_found_sum += 1
                elif k == 'size':
                    print("title: " + title_info + " | " + v)
        print("Total found " + str(title_found_sum) +
              " title of " + str(self.api.get_products_size(download_candidate)) + " GB")

        os.chdir(self.input_dir)
        if self.download:
            if glob.glob(input_dir + "*.zip") not in [value for value in download_candidate.items()]:
                self.api.download_all(download_candidate)
                print("Nothing to download")
        else:
            print("Escaping download")
        # proceed processing after download is complete
        self.sentinel1_preprocess()

    def sentinel1_preprocess(self):
        # Get snappy Operators
        snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
        # HashMap Key-Value pairs
        HashMap = snappy.jpy.get_type('java.util.HashMap')

        for folder in glob.glob(self.input_dir + "\*"):
            gc.enable()
            if folder.endswith(".zip"):
                timestamp = folder.split("_")[5]
                sentinel_image = ProductIO.readProduct(folder)
                if self.date_start <= datetime.datetime.strptime(timestamp[:8], "%Y%m%d") <= self.date_end:
                    # add orbit file
                    self.sentinel1_preprocess_orbit_file(timestamp, sentinel_image, HashMap)
                    # remove border noise
                    self.sentinel1_preprocess_border_noise(timestamp, HashMap)
                    # remove thermal noise
                    self.sentinel1_preprocess_thermal_noise_removal(timestamp, HashMap)
                    # calibrate image to output to Sigma and dB
                    self.sentinel1_preprocess_calibration(timestamp, HashMap)
                    # TOPSAR Deburst for SLC images
                    if self.producttype == 'SLC':
                        self.sentinel1_preprocess_topsar_deburst_SLC(timestamp, HashMap)
                    # multilook
                    self.sentinel1_preprocess_multilook(timestamp, HashMap)
                    # subset using a WKT of the study area
                    self.sentinel1_preprocess_subset(timestamp, HashMap)
                    # finally terrain correction, can use local data but went for the default 
                    self.sentinel1_preprocess_terrain_correction(timestamp, HashMap)
                    # break # try this if you want to check the result one by one
            
    def sentinel1_preprocess_orbit_file(self, timestamp, sentinel_image, HashMap):
        start_time_processing = datetime.datetime.now()
        orb = self.input_dir + "\\orb_" + timestamp

        if not os.path.isfile(orb + ".dim"):
            parameters = HashMap()
            orbit_param = snappy.GPF.createProduct("Apply-Orbit-File", parameters, sentinel_image)
            ProductIO.writeProduct(orbit_param, orb, 'BEAM-DIMAP')  # BEAM-DIMAP, GeoTIFF-BigTiff
            print("orbit file added: " + orb +
                  " | took: " + str(datetime.datetime.now() - start_time_processing).split('.', 2)[0])
        else:
            print("file exists - " + orb)

    def sentinel1_preprocess_border_noise(self, timestamp, HashMap):
        start_time_processing = datetime.datetime.now()
        border = self.input_dir + "\\bordr_" + timestamp

        if not os.path.isfile(border + ".dim"):
            parameters = HashMap()
            border_param = snappy.GPF.createProduct("Remove-GRD-Border-Noise", parameters,
                                                    ProductIO.readProduct(self.input_dir +
                                                                          "\\orb_" + timestamp + ".dim"))
            ProductIO.writeProduct(border_param, border, 'BEAM-DIMAP')
            print("border noise removed: " + border +
                  " | took: " + str(datetime.datetime.now() - start_time_processing).split('.', 2)[0])
        else:
            print("file exists - " + border)

    def sentinel1_preprocess_thermal_noise_removal(self, timestamp, HashMap):
        start_time_processing = datetime.datetime.now()
        thrm = self.input_dir + "\\thrm_" + timestamp

        if not os.path.isfile(thrm + ".dim"):
            parameters = HashMap()
            thrm_param = snappy.GPF.createProduct("ThermalNoiseRemoval", parameters,
                                                  ProductIO.readProduct(self.input_dir + "\\bordr_" +
                                                                        timestamp + ".dim"))
            ProductIO.writeProduct(thrm_param, thrm, 'BEAM-DIMAP')
            print("thermal noise removed: " + thrm +
                  " | took: " + str(datetime.datetime.now() - start_time_processing).split('.', 2)[0])
        else:
            print("file exists - " + thrm)

    def sentinel1_preprocess_calibration(self, timestamp, HashMap):
        start_time_processing = datetime.datetime.now()
        calib = self.input_dir + "\\calib_" + timestamp

        if not os.path.isfile(calib + ".dim"):
            parameters = HashMap()
            parameters.put('outputSigmaBand', True)
            parameters.put('outputImageScaleInDb', False)
            calib_param = snappy.GPF.createProduct("Calibration", parameters,
                                                   ProductIO.readProduct(self.input_dir + "\\thrm_" +
                                                                         timestamp + ".dim"))
            ProductIO.writeProduct(calib_param, calib, 'BEAM-DIMAP')
            print("calibration complete: " + calib +
                  " | took: " + str(datetime.datetime.now() - start_time_processing).split('.', 2)[0])
        else:
            print("file exists - " + calib)

    def sentinel1_preprocess_topsar_deburst_SLC(self, timestamp, HashMap):
        start_time_processing = datetime.datetime.now()
        deburst = self.input_dir + "\\dburs_" + timestamp

        if not os.path.isfile(deburst):
            parameters = HashMap()
            parameters.put('outputSigmaBand', True)
            parameters.put('outputImageScaleInDb', False)
            deburst_param = snappy.GPF.createProduct("TOPSAR-Deburst", parameters,
                                                     ProductIO.readProduct(self.input_dir + "\\calib_" +
                                                                           timestamp + ".dim"))
            ProductIO.writeProduct(deburst_param, deburst, 'BEAM-DIMAP')
            print("deburst complete: " + deburst +
                  " | took: " + str(datetime.datetime.now() - start_time_processing).split('.', 2)[0])
        else:
            print("file exists - " + deburst)

    def sentinel1_preprocess_multilook(self, timestamp, HashMap):
        start_time_processing = datetime.datetime.now()
        multi = self.input_dir + "\\multi_" + timestamp

        if not os.path.isfile(multi + ".dim"):
            parameters = HashMap()
            parameters.put('outputSigmaBand', True)
            parameters.put('outputImageScaleInDb', False)
            multi_param = snappy.GPF.createProduct("Multilook", parameters,
                                                   ProductIO.readProduct(self.input_dir + "\\calib_" +
                                                                         timestamp + ".dim"))
            ProductIO.writeProduct(multi_param, multi, 'BEAM-DIMAP')
            print("multilook complete: " + multi +
                  " | took: " + str(datetime.datetime.now() - start_time_processing).split('.', 2)[0])
        else:
            print("file exists - " + multi)

    def sentinel1_preprocess_subset(self, timestamp, HashMap):
        start_time_processing = datetime.datetime.now()
        subset = self.input_dir + "\\subset_" + timestamp

        if not os.path.isfile(subset + ".dim"):
            WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')
            
            # converting shapefile to GEOJSON and WKT is easy with any free online tool
            wkt = "POLYGON((92.330290184197 20.5906091141114,89.1246637610338 21.6316051481971," \
                  "89.0330319081811 21.7802436586492,88.0086282580443 24.6678836192818,88.0857830091018 " \
                  "25.9156771178278,88.1771488779853 26.1480664053835,88.3759125970998 26.5942658997298," \
                  "88.3876586919721 26.6120432770312,88.4105534167129 26.6345128356038,89.6787084683935 " \
                  "26.2383305017275,92.348481691233 25.073636976939,92.4252199249342 25.0296592837972," \
                  "92.487261172615 24.9472465376954,92.4967290851295 24.902213855393,92.6799861774377 " \
                  "21.2972058618174,92.6799346581579 21.2853347419811,92.330290184197 20.5906091141114))"

            geom = WKTReader().read(wkt)
            parameters = HashMap()
            parameters.put('geoRegion', geom)
            subset_param = snappy.GPF.createProduct("Subset", parameters,
                                                    ProductIO.readProduct(self.input_dir + "\\multi_" +
                                                                          timestamp + ".dim"))
            ProductIO.writeProduct(subset_param, subset, 'BEAM-DIMAP')
            print("subset complete: " + subset +
                  " | took: " + str(datetime.datetime.now() - start_time_processing).split('.', 2)[0])
        else:
            print("file exists - " + subset)

    def sentinel1_preprocess_terrain_correction(self, timestamp, HashMap):
        start_time_processing = datetime.datetime.now()
        terr = self.input_dir + "\\terr_" + timestamp

        if not os.path.isfile(terr + ".dim"):
            parameters = HashMap()
            # parameters.put('demResamplingMethod', 'NEAREST_NEIGHBOUR')
            # parameters.put('imgResamplingMethod', 'NEAREST_NEIGHBOUR')
            # parameters.put('pixelSpacingInMeter', 10.0)
            terr_param = snappy.GPF.createProduct("Terrain-Correction", parameters,
                                                  ProductIO.readProduct(self.input_dir + "\\subset_" +
                                                                        timestamp + ".dim"))
            ProductIO.writeProduct(terr_param, terr, 'BEAM-DIMAP')
            print("terrain corrected: " + terr +
                  " | took: " + str(datetime.datetime.now() - start_time_processing).split('.', 2)[0])
        else:
            print("file exists - " + terr)

input_dir = "/localhome/studenter/renatask/Project/testdata"
start_date = '01Mar2019'
end_date = '10Mar2019'
query_style = 'footprint' # 'footprint' to use a GEOJSON, 'coordinate' to use a lat-lon 
footprint = '/localhome/studenter/renatask/Project/testdata/map.geojson'
lat = 26.23
lon = 88.56

sar = sentinel1_download_preprocess(input_dir, start_date, end_date, query_style, footprint, lat, lon, True) 
# proceed to download by setting 'True', default is 'False'
sar.sentinel1_download()

title: S1A_IW_GRDH_1SDV_20190309T171105_20190309T171130_026264_02EF4D_0EDD | 1.58 GB
title: S1A_IW_GRDH_1SDV_20190309T171130_20190309T171155_026264_02EF4D_1310 | 1.58 GB
title: S1A_IW_GRDH_1SDV_20190309T171155_20190309T171220_026264_02EF4D_8325 | 1.58 GB
title: S1B_IW_GRDH_1SDV_20190308T171920_20190308T171945_015266_01C90B_0C57 | 1.66 GB
title: S1B_IW_GRDH_1SDV_20190308T171855_20190308T171920_015266_01C90B_D6BE | 1.66 GB
title: S1B_IW_GRDH_1SDV_20190307T163758_20190307T163827_015251_01C895_F18F | 1.83 GB
title: S1B_IW_GRDH_1SDV_20190307T163827_20190307T163852_015251_01C895_DE81 | 1.58 GB
title: S1A_IW_GRDH_1SDV_20190306T164658_20190306T164723_026220_02ED9E_2360 | 1.66 GB
title: S1A_IW_GRDH_1SDV_20190306T164633_20190306T164658_026220_02ED9E_33AD | 1.66 GB
title: S1A_IW_GRDH_1SDV_20190306T164723_20190306T164748_026220_02ED9E_E9BC | 1.66 GB
title: S1B_IW_GRDH_1SDV_20190305T165356_20190305T165421_015222_01C7A4_7AFB | 1.67 GB
title: S1B_IW_GRDH_1SDV_20190305T165421_20190305T165446_015222_01

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

INFO: org.hsqldb.persist.Logger: Database closed
